In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        (os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install -q ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.8/949.8 kB 31.2 MB/s eta 0:00:00


In [3]:
# making dir for use by YOLO
# making a dir datasets in kaggle/working and copying the data to it
!mkdir /kaggle/working/datasets
!cp -r /kaggle/input/dlp-object-detection-week-10/final_dlp_data/final_dlp_data /kaggle/working/datasets


In [4]:
#making dir for val data
!mkdir /kaggle/working/datasets/final_dlp_data/val
!mkdir /kaggle/working/datasets/final_dlp_data/val/images
!mkdir /kaggle/working/datasets/final_dlp_data/val/labels

In [5]:
#getting all labels

import os
train_dir = '/kaggle/input/dlp-object-detection-week-10/final_dlp_data/final_dlp_data/train/labels'
label_dict = {}
for file in os.listdir(train_dir):
    f = open(os.path.join(train_dir, file), 'r')
    label = f.readline().split(' ')[0]
    if label in label_dict.keys():
        label_dict[label] += 1
    else:
        label_dict[label] = 1
print(label_dict)


{'1': 3334, '3': 3312, '5': 300, '0': 35, '4': 460, '2': 59}


In [6]:
import os
import random
import shutil
count = 0
train_images_path = '/kaggle/working/datasets/final_dlp_data/train/images'
train_labels_path = '/kaggle/working/datasets/final_dlp_data/train/labels'
val_images_path = '/kaggle/working/datasets/final_dlp_data/val/images'
val_labels_path = '/kaggle/working/datasets/final_dlp_data/val/labels'
train_images = os.listdir(train_images_path)
for file in train_images:
    rand_num = random.randint(1,10)
    source_image_file = train_images_path + '/' + file
    source_label_file = train_labels_path + '/' + file.split('.')[0] + '.txt'
    dest_image_folder = val_images_path
    dest_label_folder = val_labels_path
    f = open(source_label_file, 'r')
    label = f.readline().split(' ')[0]
    if label in ['0', '2']:
        shutil.copy(source_image_file, dest_image_folder)
        shutil.copy(source_label_file, dest_label_folder)
        count += 1
    elif label in ['4','5']:
        if rand_num <= 2:
            shutil.move(source_image_file, dest_image_folder)
            shutil.move(source_label_file, dest_label_folder)
            count += 1
    else:
        if rand_num <=1:
            shutil.move(source_image_file, dest_image_folder)
            shutil.move(source_label_file, dest_label_folder)
            count += 1
    f.close()
print(count)

946


In [7]:
#Data.yaml file
f = open('/kaggle/working/data.yaml', 'w')
f.write("path: /kaggle/working/datasets/final_dlp_data # dataset root dir\n")
f.write("train: train/images # train images (relative to 'path') \n")
f.write("val: val/images # val images (relative to 'path')\n")
f.write("test: test/images # test images (optional)\n")
f.write("names:\n")
f.write("    0: aegypti\n")
f.write("    1: albopictus\n")
f.write("    2: anopheles\n")
f.write("    3: culex\n")
f.write("    4: culiseta\n")
f.write("    5: japonicus/koreicus\n")
f.close()


In [ ]:
from ultralytics import ASSETS, SAM, YOLO, FastSAM


model = YOLO("yolov8n.pt")

# Train the model
results = model.train(data="/kaggle/working/data.yaml", epochs=100, imgsz=640)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 188MB/s]


Ultralytics 8.3.95 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/kaggle/working/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True

100%|██████████| 755k/755k [00:00<00:00, 44.8MB/s]


Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...


100 epochs completed in 4.380 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.95 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 72 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 30/30 [00:13<00:00,  2.16it/s]


                   all        946        946      0.545      0.674      0.644       0.52
               aegypti         35         35      0.722      0.486      0.625      0.548
            albopictus        336        336      0.693      0.914      0.868      0.683
             anopheles         59         59      0.544      0.458       0.56      0.483
                 culex        357        357      0.682      0.944      0.896      0.682
              culiseta         97         97      0.375      0.711      0.605      0.488
    japonicus/koreicus         62         62      0.253      0.532      0.307      0.234


/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.2ms preprocess, 1.5ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/detect/train


In [9]:
import shutil
shutil.make_archive("/kaggle/working/trained_model", "zip", "/kaggle/working/runs/detect/train")
print("Zipped model saved at /kaggle/working/trained_model.zip")

Zipped model saved at /kaggle/working/trained_model.zip


In [ ]:
from ultralytics import YOLO
model = YOLO("/kaggle/working/runs/detect/train/weights/best.pt")

# Predict with the model
results = model('/kaggle/working/datasets/final_dlp_data/test/images',conf = 0.1, iou = 0.5, verbose=False)  # predict on an image

In [11]:
print(results[0])

ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'aegypti', 1: 'albopictus', 2: 'anopheles', 3: 'culex', 4: 'culiseta', 5: 'japonicus/koreicus'}
obb: None
orig_img: array([[[49, 55, 66],
        [51, 57, 68],
        [49, 58, 68],
        ...,
        [48, 62, 68],
        [49, 66, 69],
        [49, 66, 69]],

       [[44, 50, 61],
        [43, 52, 62],
        [42, 51, 61],
        ...,
        [45, 59, 65],
        [44, 61, 64],
        [42, 59, 62]],

       [[50, 58, 71],
        [53, 61, 74],
        [53, 62, 72],
        ...,
        [47, 61, 67],
        [46, 60, 66],
        [43, 57, 63]],

       ...,

       [[21, 30, 34],
        [24, 33, 37],
        [24, 33, 37],
        ...,
        [18, 22, 27],
        [17, 21, 26],
        [17, 21, 26]],

       [[25, 34, 38],
        [23, 32, 36],
        [25, 34, 38],
        ...,
        [18, 22, 27],
        [18, 22, 27],
        [18, 2

In [ ]:
import torch
#!rm /kaggle/working/submission.csv
sub_file = open('/kaggle/working/submission.csv', 'w')
sub_file.write("id,ImageID,LabelName,Conf,xcenter,ycenter,bbx_width,bbx_height\n")
counter = 0
for i, result in enumerate(results):
    if result.boxes.cls.numel() == 0:
        cls = result.names[0]
        conf = 0.0
        x = 0.0
        y = 0.0
        w = 0.0
        h = 0.0
    else:
        best_idx = torch.argmax(result.boxes.conf)
        cls = result.names[int(result.boxes.cls[best_idx].item())]
        conf = result.boxes.conf[best_idx].item()
        x = result.boxes.xywhn[best_idx][0].item()
        y = result.boxes.xywhn[best_idx][1].item()
        w = result.boxes.xywhn[best_idx][2].item()
        h = result.boxes.xywhn[best_idx][3].item()
    sub_file.write(str(counter) + ',' + os.path.basename(result.path) + ',' + str(cls) +  ',' + str(conf) + ',' + str(x) + ',' + str(y) + ',' + str(w) + ',' + str(h) +'\n' )
    counter += 1 
sub_file.close()    

In [ ]:
import pandas as pd

test_csv_path = "/kaggle/working/submission.csv" 
submission_csv_path = "/kaggle/input/dlp-object-detection-week-10/sample_submission.csv"  # Original csv format

# Load both CSV files
test_df = pd.read_csv(test_csv_path)
submission_df = pd.read_csv(submission_csv_path)

# Merge based on 'ImageID' to reorder test_df
reordered_test_df = submission_df[['ImageID']].merge(test_df, on='ImageID', how='left')

# Reorder columns to have 'id' as the first column
column_order = ['id', 'ImageID', 'LabelName', 'Conf', 'xcenter', 'ycenter', 'bbx_width', 'bbx_height']
reordered_test_df = reordered_test_df[column_order]

# Reset the 'id' column to start from 0
reordered_test_df['id'] = range(len(reordered_test_df))


test_csv_path1 = "/kaggle/working/submission.csv"
reordered_test_df.to_csv(test_csv_path1, index=False)

print(f"file saved in {test_csv_path1}")
print(reordered_test_df.head())


file saved in /kaggle/working/21f3001013_2.csv
   id                                    ImageID   LabelName      Conf  \
0   0  16dbbc39-fcb6-498e-963f-d3807fcc3cec.jpeg  albopictus  0.855454   
1   1  7daa68bd-928b-4b43-850f-4626f16d2c14.jpeg  albopictus  0.904879   
2   2  bb0986d9-fa04-40a5-8de2-c54045537e17.jpeg  albopictus  0.758790   
3   3  11924d04-69fe-469d-8d1c-119bc0f0b94f.jpeg       culex  0.691447   
4   4  db4ade79-fce4-4d77-a8fa-36b9e77dfd08.jpeg       culex  0.446287   

    xcenter   ycenter  bbx_width  bbx_height  
0  0.432325  0.450116   0.133256    0.111599  
1  0.497844  0.556036   0.430074    0.300286  
2  0.360669  0.472824   0.112291    0.173380  
3  0.630166  0.627201   0.735247    0.745597  
4  0.363856  0.471260   0.180790    0.123030  


In [ ]:
df = pd.read_csv("/kaggle/working/submission.csv")
df

,id,ImageID,LabelName,Conf,xcenter,ycenter,bbx_width,bbx_height
0,0,16dbbc39-fcb6-498e-963f-d3807fcc3cec.jpeg,albopictus,0.855454,0.432325,0.450116,0.133256,0.111599
1,1,7daa68bd-928b-4b43-850f-4626f16d2c14.jpeg,albopictus,0.904879,0.497844,0.556036,0.430074,0.300286
2,2,bb0986d9-fa04-40a5-8de2-c54045537e17.jpeg,albopictus,0.758790,0.360669,0.472824,0.112291,0.173380
3,3,11924d04-69fe-469d-8d1c-119bc0f0b94f.jpeg,culex,0.691447,0.630166,0.627201,0.735247,0.745597
4,4,db4ade79-fce4-4d77-a8fa-36b9e77dfd08.jpeg,culex,0.446287,0.363856,0.471260,0.180790,0.123030
...,...,...,...,...,...,...,...,...
520,520,b719ddba-3666-4dfd-848b-afe465d4dd5c.jpeg,albopictus,0.887433,0.436117,0.653072,0.296482,0.245624
521,521,87573bc0-f7fd-4da6-b085-c331d740039f.jpeg,albopictus,0.865109,0.480667,0.476263,0.153525,0.128223
522,522,3a918cb9-770c-4e13-8201-90858c25c7d0.jpeg,culex,0.665645,0.361647,0.544180,0.116143,0.078291
523,523,da0226ce-ef8f-4217-bd5d-38955a232eab.jpeg,culiseta,0.867244,0.480502,0.504127,0.818324,0.935501
